# Model Formulation

This notebook can be used to run initial model with DCOPF and a basic budget variable. Before running this nodebook please refer to `Test-System-Data-Processing.ipynb` to preprocess generator, lines, buses, and loads data. 

In [2]:
using JuMP, Gurobi
using DataFrames, CSV

include(joinpath("capex_model", "expansion_tools.jl"))

save (generic function with 1 method)

In [14]:
function basic_capex(buses, lines, gens, loads)
    """
    Function to solve DC OPF problem 
    Inputs:
        gens -- dataframe with generator info and costs
        lines -- dataframe with transmission lines info
        buses -- dataframe with bus types and loads
    """   
    #### HELPER FUNCTIONS ####
    # Select a line column using i and j 
    line = (i, j, col) -> lines[lines_map[(i, j)], col]
    # Subset of all lines connected to bus i
    J = i -> lines.t_bus[lines.f_bus .== i]
    # Select all generators in G connected to bus i
    sel = (G, i) -> G[gens[G,:bus] .== i]
    
    ##### SETS ####
    G = 1:nrow(gens)             # G: Set of all generators
    G_s = G[gens.ess]            # Gₛ: Set of all energy storage system (ESS) generators
    
    N = 1:nrow(buses)            # N: Set of all network nodes
    L = collect(zip(             # L: Set of all lines/branches
        lines.f_bus, lines.t_bus))
    
    T = 1:ncol(loads)            # T: Set of all hours
    P = [T]                      # P: Set of representative periods

    #### GLOBALS ####
    # Mapping from (i, j) tuple to line index l
    lines_map = Dict(zip(L, 1:length(L)))
    
    #### PARAMETERS ####
    θlim = π*(60/180)  # Absolute max angle limit in rad
    slack_bus = 416    # Set slack bus to one with many CTs
    
    storage_hrs = 4    # Storage Hours * Power Capacity = Energy capacity ∀ g ∈ G_s
    η_charge = 0.97    # Charging efficiency  
    η_discharge = 0.91 # Discharging efficiecy

    # Toy parameters for a limited investment in solar at each bus
    shed_cost = 1000   # Cost of load sheding
    budget = 0
    solar_cost = 0.1475
    
    ########################
    #####     MODEL    #####
    ########################
    
    Expansion_Model = Model(() -> Gurobi.Optimizer())
    #set_optimizer_attribute(Expansion_Model, "NumericFocus", 2)
    
    # Decision variables   
    @variables(Expansion_Model, begin
        GEN[G,T]           # Generation of each generator 
        SHED[N,T] ≥ 0      # Load sheading at bus N
        THETA[N,T]         # Voltage phase angle of bus
        FLOW[L,T]          # Flows between all pairs of nodes
        SOC[G_s,T] ≥ 0     # ESS state of charge
        CHARGE[G_s,T] ≥ 0  # Chargeing of ESS 
    end)
    
    # Objective function (Note: Using just the linear part of quadratic cost)
    @objective(Expansion_Model, Min,
            sum(gens[g,:c0] + gens[g,:c1] * GEN[g,t] for g ∈ G, t ∈ T) 
            + sum(shed_cost * SHED[i,t] for i ∈ N, t ∈ T) #TODO expressions
    ) 
    
    # Supply demand balances: sum(generation) + shedding - demand - sum(charging) = sum(flows)
    @constraint(Expansion_Model, cBalance[i ∈ N, t ∈ T], 
        sum(GEN[g,t] for g ∈ sel(G, i))
        + SHED[i,t] - loads[i,t] 
        - sum(CHARGE[g,t] for g ∈ sel(G_s, i))
         == sum(FLOW[(i,j),t] for j ∈ J(i))
    )
    
    # Max and min generation constraints
    @constraint(Expansion_Model, cMaxGen[g ∈ G, t ∈ T], GEN[g,t] ≤ gens[g,:pmax]) ## TODO add capacity factors
    @constraint(Expansion_Model, cMinGen[g ∈ G, t ∈ T], GEN[g,t] ≥ gens[g,:pmin])

    # Load shedding constraint
    @constraint(Expansion_Model, cShed[i ∈ N, t ∈ T], SHED[i,t] ≤ loads[i,t])

    #### BATTERY STORAGE ####
    
    # Energy storage and charging constraints
    @constraint(Expansion_Model, cMaxCharge[g ∈ G_s, t ∈ T], CHARGE[g,t] ≤ gens[g,:pmax])
    @constraint(Expansion_Model, cMaxStateOfCharge[g ∈ G_s, t ∈ T], SOC[g,t] ≤ storage_hrs*gens[g,:pmax])

    for P_i ∈ P
        # Number of hours in the representative time period Pᵢ
        p = length(P_i)
        # Recursively constrain all SOCs with wrapping for each period
        @constraint(Expansion_Model, cStateOfCharge[g ∈ G_s, t ∈ P_i],
            SOC[g,t] == SOC[g,mod1(t-1,p)] 
            + (CHARGE[g,t]*η_charge - GEN[g,t]/η_discharge)
        )
    end
    
    #### DC OPF ####
            
    # Create slack bus with theta=0
    fix.(THETA[slack_bus, T], 0)
    
    # Max line flow constraints
    @constraint(Expansion_Model, cLineLimits[(i,j) ∈ L, t ∈ T], 
        FLOW[(i,j),t] ≤ line(i,j,:rate_a)) 
    
    # Angle limits 
    @constraint(Expansion_Model, cAngleLimitsMax[(i,j) ∈ L, t ∈ T], 
        (THETA[i,t] - THETA[j,t]) ≤  θlim)
    @constraint(Expansion_Model, cAngleLimitsMin[(i,j) ∈ L, t ∈ T], 
        (THETA[i,t] - THETA[j,t]) ≥ -θlim)
                    
    # Flow constraints on each branch
    @constraint(Expansion_Model, cLineFlows[(i,j) ∈ L, t ∈ T],
        FLOW[(i,j),t] == line(i,j,:sus)*(THETA[i,t] - THETA[j,t]));

    optimize!(Expansion_Model)

    return Expansion_Model
end

basic_capex (generic function with 1 method)

In [16]:
# 2. Capacity factors --> ask Qi about how to use CF datasets (are added nodes treated differently?)

# 3b. Ramping
# 4. Investment decisions
# 5. Representative periods


## Modules



## EXTRA
# Quardatic costs


In [18]:
run_model(load, basic_capex, save, "san_diego_system", "test_senario")

Set parameter Username
Set parameter LicenseID to value 2669913
Academic license - for non-commercial use only - expires 2026-05-22
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M2 Ultra
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 147960 rows, 56832 columns and 298488 nonzeros
Model fingerprint: 0x04e40ec5
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [7e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 3e+03]
Presolve removed 127260 rows and 35292 columns
Presolve time: 0.11s
Presolved: 20700 rows, 25928 columns, 91760 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.02s

Barrier statistics:
 Free vars  : 882
 AA' NZ     : 1.980e+05
 Factor NZ  : 5.392e+05 (roughly 23 MB of memory)
 Factor Ops : 1.770e+07 (less than 1 second per iteration)
 Thread

"/Users/adamsedlak/Documents/Julia/MAE243/final-project/models/san_diego_system/outputs/test_senario/prices.csv"

In [8]:
buses, lines, gens, loads = load("san_diego_system");